In [1]:
import os
import sys
import time
import glob
import numpy as np
import logging
import argparse
import tensorflow as tf
import numpy as np
import utils
from tqdm import tqdm
import shutil
from model_search import Network
from architect_graph import Architect

import time
from genotypes import PRIMITIVES
from genotypes import Genotype
from scipy.special import softmax

In [2]:
# tf.enable_eager_execution()
tf.set_random_seed(6969)
np.random.seed(6969)

In [3]:
import utils
from genotypes import Genotype

In [4]:
args = {
    "momentum": 0.9,
    "weight_decay": 3e-4,
    "arch_learning_rate": 3e-1,
    "momentum": 0.9,
    "grad_clip": 5,
    "learning_rate": 0.025,
    "learning_rate_decay": 0.97,
    "learning_rate_min": 0.0001,
    "num_batches_per_epoch": 2000,
    
    "unrolled": True,
    "epochs": 10,
    "train_batch_size": 8,
    "eval_batch_size": 8,
    "save": "EXP",
    "init_channels": 3,
    "num_layers": 3,
    "num_classes": 6,
    "crop_size": [8, 8],
    "save_checkpoints_steps": 100,
    "model_dir": 'gs://unet-darts/train-search-ckptss',
    "max_steps": 10000,
    # NEW
    "steps_per_eval": 2,
    "num_train_examples": 16,
    "num_batches_per_epoch": 2,
    #
    
    "use_tpu": False,
    "use_host_call": False,
    "tpu": 'unet-darts',
#     "zone": 'us-central1-f',
#     "project": "isro-nas"
    "zone": None,
    "project": None
}

class Struct:
    def __init__(self, **entries):
        self.__dict__.update(entries)

args = Struct(**args)

### input_fn

In [5]:
def make_inp_fn(filename, mode, batch_size):
    
    def _input_fn(params):
        image_dataset = tf.data.TFRecordDataset(filename)
        W, H = args.crop_size[0], args.crop_size[1]

        # Create a dictionary describing the features.  
        image_feature_description = {
            'train_name': tf.FixedLenFeature([], tf.string),  
            'train_x': tf.FixedLenFeature([], tf.string),
            'train_y': tf.FixedLenFeature([], tf.string),
            'valid_name': tf.FixedLenFeature([], tf.string),  
            'valid_x': tf.FixedLenFeature([], tf.string),
            'valid_y': tf.FixedLenFeature([], tf.string)
        }
        def _parse_image_function(example_proto):
            # Parse the input tf.Example proto using the dictionary above.
            feature= tf.parse_single_example(example_proto, image_feature_description)
            train_x = feature['train_x']
            train_y = feature['train_y']
            valid_x = feature['valid_x']
            valid_y = feature['valid_y']
            train_name = feature['train_name']
            valid_name = feature['valid_name']

            train_x = tf.image.decode_png(train_x, channels=3)
            train_y = tf.image.decode_png(train_y, channels=3)
            valid_x = tf.image.decode_png(valid_x, channels=3)
            valid_y = tf.image.decode_png(valid_y, channels=3)
            
            
            train_x = tf.cast(train_x, tf.float32)
            train_x = tf.image.resize(train_x, (W, H))
            train_y = tf.cast(train_y, tf.float32)
            train_y = tf.image.resize(train_y, (W, H))
            valid_x = tf.cast(valid_x, tf.float32)
            valid_x = tf.image.resize(valid_x, (W, H))
            valid_y = tf.cast(valid_y, tf.float32)
            valid_y = tf.image.resize(valid_y, (W, H))
            
            train_y = train_y[:, :, 0]
            train_y = tf.expand_dims(train_y, axis=-1)
            
            valid_y = valid_y[:, :, 0]
            valid_y = tf.expand_dims(valid_y, axis=-1)
            
            return ((train_x, valid_x), (train_y, valid_y))

        dataset = image_dataset.map(_parse_image_function)
        
        if mode == tf.estimator.ModeKeys.TRAIN:
            num_epochs = None # indefinitely
            dataset = dataset.shuffle(buffer_size = 10 * batch_size)
        else:
            num_epochs = 1 # end-of-input after this

        dataset = dataset.repeat(num_epochs).batch(batch_size,  drop_remainder=True)

        return dataset
    
    return _input_fn

In [74]:
def make_inp_fn2(filename, mode, batch_size):
    
    def _input_fn(params):
        W, H = args.crop_size[0], args.crop_size[1]
        NUM_IMAGES = 20
        x_train = np.random.randint(0, 256, (NUM_IMAGES, W, H, 3)).astype(np.float32)
        y_train = np.random.randint(0, args.num_classes, (NUM_IMAGES, W, H, 1)).astype(np.float32)
        x_valid = np.random.randint(0, 256, (NUM_IMAGES, W, H, 3)).astype(np.float32)
        y_valid = np.random.randint(0, args.num_classes, (NUM_IMAGES, W, H, 1)).astype(np.float32)

        ds = (x_train, x_valid), (y_train, y_valid)
        dataset = tf.data.Dataset.from_tensor_slices(ds)
        num_epochs = None # indefinitely
        dataset = dataset.shuffle(buffer_size = 10 * batch_size)

        dataset = dataset.repeat(num_epochs).batch(batch_size, drop_remainder=True)
        return dataset
    
    return _input_fn

### Model function

In [75]:
class GeneSaver(tf.estimator.SessionRunHook):
    def __init__(self, model, alphas_normal, alphas_reduce):
        self.model = model
        self.alphas_normal = alphas_normal
        self.alphas_reduce = alphas_reduce
    
    def begin(self):
        self.global_step = tf.train.get_or_create_global_step()
        
    def end(self, session):
        alphas_normal, alphas_reduce = session.run([self.alphas_normal, self.alphas_reduce])
        alphas_normal = softmax(alphas_normal)
        alphas_reduce = softmax(alphas_reduce)
        
        genotype = self.model.genotype(alphas_normal, alphas_reduce)        
        self.global_step = session.run(self.global_step)
        
        filename = 'final_genotype.{}'.format((self.global_step))
        tf.logging.info("Saving Genotype for step: {}".format(str(self.global_step)))
        utils.write_genotype(genotype, filename)

In [85]:
def model_fn(features, labels, mode, params):
    criterion = tf.losses.softmax_cross_entropy
    model = Network(C=args.init_channels, net_layers=args.num_layers, criterion=criterion, num_classes=args.num_classes)
#     _ = model(np.random.randn(args.train_batch_size))
    eval_hooks = None
    loss = None
    train_op = None
    eval_metric_ops = None
    prediction_dict = None
    export_outputs = None
    host_call = None
    
    # 2. Loss function, training/eval ops
    if mode == tf.estimator.ModeKeys.TRAIN:
        global_step = tf.train.get_global_step()
        learning_rate_min = tf.constant(args.learning_rate_min)

        learning_rate = tf.train.exponential_decay(
            args.learning_rate,
            global_step,
            decay_rate=args.learning_rate_decay,
            decay_steps=args.num_batches_per_epoch,
            staircase=True,
        )

        lr = tf.maximum(learning_rate, learning_rate_min)

        optimizer = tf.train.MomentumOptimizer(lr, args.momentum)

        if(args.use_tpu):
            optimizer = tf.tpu.CrossShardOptimizer(optimizer)
        (x_train, x_valid) = features
        (y_train, y_valid) = labels

        preds = model(x_train)
#         architect = Architect(model, args)
#        # architect step
#         architect_step = architect.step(input_train=x_train,
#                                         target_train=y_train,
#                                         input_valid=x_valid,
#                                         target_valid=y_valid,
#                                         unrolled=args.unrolled,
#                                         )


#         with tf.control_dependencies([architect_step]):
        update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
        with tf.control_dependencies(update_ops):
            w_var = model.get_thetas()
            loss = model._loss(preds, y_train)
            grads = tf.gradients(loss, w_var, name='train_gradients')
            train_op = optimizer.apply_gradients(zip(grads, w_var), global_step=tf.train.get_global_step())

        b, w, h, c = y_train.shape
        y = tf.reshape(tf.cast(y_train, tf.int64), (b, w, h))
        y = tf.one_hot(y, args.num_classes, on_value=1.0, off_value=0.0)
        
        if args.use_host_call:
            def host_call_fn(global_step, learning_rate, loss, y, preds):
                # Outfeed supports int32 but global_step is expected to be int64.
                global_step = tf.reduce_mean(global_step)
                global_step = tf.cast(global_step, tf.int64)
                miou, conf_mat = tf.metrics.mean_iou(
                    labels=y,
                    predictions=tf.round(preds),
                    num_classes=args.num_classes)
                
                acc = tf.metrics.accuracy(labels=y, 
                                              predictions=tf.round(preds))
                with (tf.contrib.summary.create_file_writer(args.model_dir).as_default()):
                    with tf.contrib.summary.always_record_summaries():
                        tf.contrib.summary.scalar(
                            'learning_rate', tf.reduce_mean(learning_rate),
                            step=global_step)
                        tf.contrib.summary.scalar(
                            'loss', tf.reduce_mean(loss),step=global_step)
                        tf.contrib.summary.scalar(
                            'acc', acc,step=global_step)
                        tf.contrib.summary.scalar(
                            'miou', miou, step=global_step)
                return tf.contrib.summary.all_summary_ops()

            global_step_t = tf.reshape(global_step, [1])
            learning_rate_t = tf.reshape(learning_rate, [1])
            loss_t = tf.reshape(loss, [1])
            host_call = (host_call_fn,
                           [global_step_t, learning_rate_t, loss_t, y, preds])

    elif mode == tf.estimator.ModeKeys.EVAL:
        alphas_normal = model.arch_parameters()[0]
        alphas_reduce = model.arch_parameters()[1]
        gene_saver = GeneSaver(model, alphas_normal, alphas_reduce)
        eval_hooks = [gene_saver]
        
        (x_train, x_valid) = features
        (y_train, y_valid) = labels
        preds = model(x_valid)
        loss = model._loss(preds, y_valid)
        
        b, w, h, c = y_valid.shape
        y = tf.reshape(tf.cast(y_valid, tf.int64), (b, w, h))
        y = tf.one_hot(y, args.num_classes, on_value=1.0, off_value=0.0)
        
        metric_fn = lambda y, preds: {
            'miou': tf.metrics.mean_iou(
                    labels=y,
                    predictions=tf.round(preds),
                    num_classes=args.num_classes
                ),
        
            'acc': tf.metrics.accuracy(labels=y, 
                                      predictions=tf.round(preds)),
#             'loss': loss
        }
        eval_metric_ops = (metric_fn, [y, preds])
        
    elif mode == tf.estimator.ModeKeys.PREDICT:
        x = features
        y = labels
        preds = model(x)
        prediction_dict = {"predictions": preds}
        export_outputs = {"predict_export_outputs": tf.estimator.export.PredictOutput(outputs = preds)}
    
    # 5. Return EstimatorSpec
    return tf.estimator.tpu.TPUEstimatorSpec(
        mode = mode,
        predictions = prediction_dict,
        loss = loss,
        train_op = train_op,
        eval_metrics = eval_metric_ops,
        export_outputs = export_outputs,
        evaluation_hooks=eval_hooks,
        host_call=host_call
    )

### Estimator

In [86]:
# Create functions to read in respective datasets
def get_train():
    return make_inp_fn2(filename = tf.gfile.Glob('../../datasets/train_search/train-search-*-00008.tfrecords'),
                        mode = tf.estimator.ModeKeys.TRAIN,
                        batch_size = args.train_batch_size)

In [87]:
# Create serving input function
def serving_input_fn():
    feature_placeholders = {
      IMAGE_LOC: tf.placeholder(tf.float32, [None])
    }
    
    feature_placeholders['IMAGES'] = tf.placeholder(tf.float32, [None, args.crop_size[0], args.crop_size[1], args.init_channels])
    
    features = {
    key: tf.expand_dims(tensor, -1)
    for key, tensor in feature_placeholders.items()
    }

    return tf.estimator.export.ServingInputReceiver(features, feature_placeholders)

In [88]:
tpu_cluster_resolver = tf.distribute.cluster_resolver.TPUClusterResolver(
      tpu=args.tpu,
      zone=args.zone,
      project=args.project)

In [89]:
config = tf.estimator.tpu.RunConfig(
      cluster=tpu_cluster_resolver,
      model_dir=args.model_dir,
      save_checkpoints_steps=args.save_checkpoints_steps,
      tpu_config=tf.contrib.tpu.TPUConfig(num_shards=8))

In [90]:
estimator = tf.estimator.tpu.TPUEstimator(
    use_tpu=args.use_tpu,
    model_fn=model_fn,
    config=config,
    train_batch_size=args.train_batch_size,
    eval_batch_size=args.eval_batch_size,
    params=vars(args)
)

INFO:tensorflow:Using config: {'_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.240.1.18:8470"
    }
  }
}
isolate_session_state: true
, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_train_distribute': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f5ea3390f10>, '_model_dir': 'gs://unet-darts/train-search-ckptss', '_protocol': None, '_save_checkpoints_steps': 100, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tpu_config': TPUConfig(iterations_per_loop=2, num_shards=8, num_cores_per_replica=None, per_host_input_for_training=2, tpu_job_name=None, initial_infeed_sleep_secs=None, input_partition_dims=None, eval_training_input_configuration=2, experimental_host_call_every_n_steps=1), '_tf_random_seed': None, '_save_summary_steps': 100, '_device_fn': None, '_session_creation_time

In [91]:
class AlphasPrinter(tf.train.SessionRunHook):
    def __init__(self):
        pass
    
    def begin(self):
        self.global_step = tf.train.get_or_create_global_step()
        
    def _get_alphas(self):
        specific_tensor = []
        specific_tensor_name = []
        for var in tf.trainable_variables():
            if not 'alphas' in var.name:
                specific_tensor.append(var)
                specific_tensor_name.append(var.name)
        return [specific_tensor[0]], [specific_tensor_name[0]]
    
    def end(self, session):
        current_step = session.run(self.global_step)
        alphas = self._get_alphas()[0]
        alphas_o = session.run(alphas)
        tf.logging.info("Printing alphas \n")
        names = self._get_alphas()[1]
        tf.logging.info(list(zip(names, alphas_o)))
        tf.logging.info("\n")

In [93]:
alphas_printer = AlphasPrinter()
estimator.train(input_fn = get_train(), steps=20, hooks=[alphas_printer])
# Loss for final step: 1.9186516.
# INFO:tensorflow:[('network/sequential/conv2d/kernel:0', array([[[[-3.13719720e-01, -2.10635364e-01,  1.38333917e-01,

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Running train on CPU
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from gs://unet-darts/train-search-ckptss/model.ckpt-970
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 970 into gs://unet-darts/train-search-ckptss/model.ckpt.
INFO:tensorflow:Saving checkpoints for 990 into gs://unet-darts/train-search-ckptss/model.ckpt.
INFO:tensorflow:Printing alphas 

INFO:tensorflow:[('network/sequential/conv2d/kernel:0', array([[[[-3.13719720e-01, -2.10635364e-01,  1.38333917e-01,
           3.39387923e-01, -1.60747975e-01,  1.60995185e-01,
          -1.68886140e-01,  9.87251550e-02, -1.93222836e-01],
         [ 8.36660713e-02, -1.48222148e-01, -6.59976080e-02,
           2.71289200e-01,  1.66718140e-02,  2.15990886e-01,
           2.59747684e-01, -1.11556143e-01, -

In [63]:
estimator.get_variable_names()

['conv2d/kernel',
 'conv2d/kernel/Momentum',
 'conv2d_1/kernel',
 'conv2d_1/kernel/Momentum',
 'conv2d_10/kernel',
 'conv2d_10/kernel/Momentum',
 'conv2d_100/kernel',
 'conv2d_100/kernel/Momentum',
 'conv2d_101/kernel',
 'conv2d_101/kernel/Momentum',
 'conv2d_102/kernel',
 'conv2d_102/kernel/Momentum',
 'conv2d_103/kernel',
 'conv2d_103/kernel/Momentum',
 'conv2d_104/kernel',
 'conv2d_104/kernel/Momentum',
 'conv2d_105/kernel',
 'conv2d_105/kernel/Momentum',
 'conv2d_106/kernel',
 'conv2d_106/kernel/Momentum',
 'conv2d_107/kernel',
 'conv2d_107/kernel/Momentum',
 'conv2d_108/kernel',
 'conv2d_108/kernel/Momentum',
 'conv2d_109/kernel',
 'conv2d_109/kernel/Momentum',
 'conv2d_11/kernel',
 'conv2d_11/kernel/Momentum',
 'conv2d_110/kernel',
 'conv2d_110/kernel/Momentum',
 'conv2d_111/kernel',
 'conv2d_111/kernel/Momentum',
 'conv2d_112/kernel',
 'conv2d_112/kernel/Momentum',
 'conv2d_113/kernel',
 'conv2d_113/kernel/Momentum',
 'conv2d_114/kernel',
 'conv2d_114/kernel/Momentum',
 'conv2d_

In [54]:
estimator.get_variable_value('network/sequential/conv2d/kernel:0')

array([[[[-3.13719720e-01, -2.10635364e-01,  1.38333917e-01,
           3.39387923e-01, -1.60747975e-01,  1.60995185e-01,
          -1.68886140e-01,  9.87251550e-02, -1.93222836e-01],
         [ 8.36660713e-02, -1.48222148e-01, -6.59976080e-02,
           2.71289200e-01,  1.66718140e-02,  2.15990886e-01,
           2.59747684e-01, -1.11556143e-01, -1.71548184e-02],
         [-1.15309805e-01, -1.01198070e-01,  7.82530010e-02,
           1.33930445e-01,  8.42399299e-02,  1.42105207e-01,
           1.29673615e-01, -1.32413516e-02, -3.27356756e-02]],

        [[-3.66295017e-02, -5.70783243e-02,  2.88080454e-01,
           1.81272760e-01,  2.16118053e-01,  1.60589665e-01,
           7.94534236e-02, -9.77155864e-02, -9.80683491e-02],
         [ 5.20635694e-02, -2.41009116e-01,  2.20164493e-01,
          -1.39509052e-01,  1.60952106e-01,  1.41411439e-01,
          -1.96783587e-01, -2.34680533e-01, -1.19687259e-01],
         [-1.46452323e-01,  1.94279477e-01,  2.51671493e-01,
           3.2093

In [62]:
estimator.get_variable_value('network/alphas_normal')

array([[5.5880670e-04, 4.2701152e-04, 3.7361731e-04, 9.0011110e-04],
       [3.0350615e-04, 3.5084834e-04, 5.9144740e-04, 1.9350863e-04],
       [6.4836361e-04, 7.9942564e-04, 3.6264589e-04, 3.6002375e-04],
       [7.5876858e-04, 6.0126593e-04, 7.3827698e-04, 8.9650316e-04],
       [6.3201052e-04, 5.8899715e-04, 3.1883040e-04, 6.5328507e-04],
       [5.8266270e-04, 3.8731575e-04, 2.6149786e-04, 8.5033552e-04],
       [5.0282837e-05, 6.9194857e-04, 2.5619520e-04, 9.9307508e-04],
       [2.4402405e-04, 6.0913770e-04, 6.0620074e-05, 6.8362453e-04],
       [9.9648500e-04, 4.6333743e-04, 9.5534354e-04, 8.2748942e-04],
       [3.7351003e-04, 5.4032507e-04, 1.6283990e-04, 5.1641965e-04],
       [1.3891865e-04, 8.3427993e-04, 6.1086001e-04, 3.7770893e-04],
       [2.8008211e-04, 7.2481995e-04, 4.5247914e-05, 8.4861997e-04],
       [2.7125169e-04, 6.9532613e-04, 6.4367324e-04, 2.5386811e-04],
       [8.2803826e-04, 8.4666198e-04, 4.4704677e-04, 5.9973681e-04]],
      dtype=float32)

In [57]:
alphas_printer = AlphasPrinter()
estimator.evaluate(input_fn = get_train(), steps=1, hooks=[alphas_printer])
#  acc = 0.7265625, global_step = 500, loss = 1.9342766, miou = 0.40841663
# INFO:tensorflow:[('network/sequential/conv2d/kernel:0', array([[[[-3.13719720e-01, -2.10635364e-01,  1.38333917e-01,

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-12-07T11:19:27Z
INFO:tensorflow:TPU job name worker
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from gs://unet-darts/train-search-ckptss/model.ckpt-900
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Init TPU system
INFO:tensorflow:Initialized TPU in 8 seconds
INFO:tensorflow:Starting infeed thread controller.
INFO:tensorflow:Starting outfeed thread controller.
INFO:tensorflow:Initialized dataset iterators in 0 seconds
INFO:tensorflow:Enqueue next (1) batch(es) of data to infeed.
INFO:tensorflow:Dequeue next (1) batch(es) of data from outfeed.
INFO:tensorflow:Outfeed finished for iteration (0, 0)
INFO:tensorflow:Evaluation [1/1]
INFO:tensorflow:Printing alphas 

INFO:tensorflow:[('network/sequential/conv2d/kernel:0', array([[[[-3.13719720e-01, -2.10635364e-01,  1.38333917e-01,
           3.3

{'acc': 0.71809894, 'global_step': 900, 'loss': 1.8717766, 'miou': 0.39712805}

## Extra Test

In [14]:
criterion = tf.losses.softmax_cross_entropy
model = Network(C=args.init_channels, net_layers=args.num_layers, criterion=criterion, num_classes=args.num_classes)

In [15]:
(x_train, x_valid), (y_train, y_valid) = get_train()(vars(args)).make_one_shot_iterator().get_next()

Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_one_shot_iterator(dataset)`.


Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_one_shot_iterator(dataset)`.


In [16]:
logits = model(x_train)

In [14]:
loss = model._loss(logits, y_train)

(<tf.Tensor 'one_hot:0' shape=(8, 4, 4, 6) dtype=float32>, <tf.Tensor 'network/softmaxConv/softmax/Softmax:0' shape=(8, 4, 4, 6) dtype=float32>)


In [16]:
optimizer = tf.train.GradientDescentOptimizer(0.001)
optimizer.minimize(loss, var_list=model.get_thetas())

<tf.Operation 'GradientDescent' type=NoOp>

In [17]:
y_train

<tf.Tensor 'IteratorGetNext:2' shape=(8, 4, 4, 1) dtype=float32>

In [18]:
logits

<tf.Tensor 'network/softmaxConv/softmax/Softmax:0' shape=(8, 4, 4, 6) dtype=float32>

In [20]:
b, w, h, c = y_train.shape
y = tf.reshape(tf.cast(y_train, tf.int64), (b, w, h))
y = tf.one_hot(y, args.num_classes, on_value=1.0, off_value=0.0)

In [21]:
y

<tf.Tensor 'one_hot:0' shape=(8, 4, 4, 6) dtype=float32>

In [53]:
miou = tf.metrics.mean_iou(
    labels=y,
    predictions=tf.round(logits),
    num_classes=args.num_classes
)

In [50]:
b = tf.metrics.mean_iou(
    labels=tf.constant([[1, 0, 0], [0, 1, 0]]),
    predictions=tf.constant([[1, 0, 0], [0, 0.99999, 0]]),
    num_classes=2
)

In [54]:
local_op = tf.local_variables_initializer()
global_op = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run([local_op, global_op])
    out = sess.run(miou)